In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, year, to_date, count, sum as _sum, expr, rank
from pyspark.sql.window import Window
from time import time

#Αρχικοποίηση SparkSession
spark = SparkSession.builder.appName("Query 2a - Top 3 Police Stations").getOrCreate()

#Φόρτωση των δεδομένων από αρχεία CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'  
file_path2 = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
data1 = spark.read.csv(file_path, header=True, inferSchema=True)
data2 = spark.read.csv(file_path2, header=True, inferSchema=True)

#Ένωση των δύο συνόλων δεδομένων
data = data1.union(data2)

#Χρόνος εκτέλεσης για το DataFrame API
df_start = time()

#Eξαγωγή του έτους από τη στήλη 'Date Rptd'
data = data.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))

#Προσθήκη στήλης που ελέγχει αν η υπόθεση έχει κλείσει (δηλαδή δεν είναι "Invest Cont ή UNK")
data = data.withColumn("is_closed",when(~col("Status Desc").rlike("Invest Cont|UNK"), 1).otherwise(0))

#Ομαδοποίηση κατά έτος και όνομα περιοχής για υπολογισμό συνολικών και κλειστών υποθέσεων
summary = data.groupBy("year", "AREA NAME").agg(
    count("DR_NO").alias("total_cases"),  #Συνολικές υποθέσεις
    _sum("is_closed").alias("closed_cases")  #Κλειστές υποθέσεις
)

#Υπολογισμός ποσοστού κλειστών υποθέσεων
summary = summary.withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

#Ορισμός παραθύρου για κατάταξη με βάση το ποσοστό κλειστών υποθέσεων
window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

#Προσθήκη κατάταξης με βάση το ποσοστό κλειστών υποθέσεων
summary = summary.withColumn("rank", rank().over(window_spec))

#Φιλτράρισμα για τις 3 καλύτερες αστυνομικές περιοχές ανά έτος
top3_summary = summary.filter(col("rank") <= 3)

#Ταξινόμηση αποτελεσμάτων κατά έτος και κατάταξη
top3_summary = top3_summary.orderBy("year", "rank")

df_end = time()

#Μέτρηση χρόνου εκτέλεσης για το DataFrame API
df_time = df_end - df_start

#Εμφάνιση αποτελεσμάτων
top3_summary.select("year", "AREA NAME", "closed_case_rate", "rank").show(top3_summary.count(), truncate=False)

#Χρόνος εκτέλεσης για SQL API
sql_start = time()

#Δημιουργία προσωρινής προβολής για το σύνολο δεδομένων
data.createOrReplaceTempView("crime_data")

#Εξαγωγή έτους από 'Date Rptd' και προσθήκη στήλης για έλεγχο αν η υπόθεση έχει κλείσει
spark.sql("""
    CREATE OR REPLACE TEMP VIEW processed_data AS
    SELECT *,
           YEAR(TO_DATE(`Date Rptd`, 'MM/dd/yyyy hh:mm:ss a')) AS years,
           CASE WHEN `Status Desc` NOT RLIKE 'Invest Cont|UNK' THEN 1 ELSE 0 END AS closed
    FROM crime_data
""")

#Ομαδοποίηση κατά έτος και όνομα περιοχής για υπολογισμό συνολικών και κλειστών υποθέσεων
spark.sql("""
    CREATE OR REPLACE TEMP VIEW summary AS
    SELECT years,
           `AREA NAME`,
           COUNT(DR_NO) AS total_cases,  -- Συνολικές υποθέσεις
           SUM(closed) AS closed_cases,  -- Κλειστές υποθέσεις
           (SUM(closed) / COUNT(DR_NO)) * 100 AS closed_case_rate  -- Ποσοστό κλειστών υποθέσεων
    FROM processed_data
    GROUP BY years, `AREA NAME`
""")

#Ορισμός παραθύρου για κατάταξη και προσθήκη κατάταξης με βάση το ποσοστό κλειστών υποθέσεων
spark.sql("""
    CREATE OR REPLACE TEMP VIEW ranked_summary AS
    SELECT *,
           RANK() OVER (PARTITION BY years ORDER BY closed_case_rate DESC) AS rank
    FROM summary
""")

#Φιλτράρισμα για τις 3 καλύτερες αστυνομικές περιοχές ανά έτος και ταξινόμηση αποτελεσμάτων
top3_summary_sql = spark.sql("""
    SELECT years, `AREA NAME`, closed_case_rate, rank
    FROM ranked_summary
    WHERE rank <= 3
    ORDER BY years, rank
""")

sql_end = time()

#Εμφάνιση αποτελεσμάτων
top3_summary_sql.show(top3_summary_sql.count(), truncate=False)

#Μέτρηση χρόνου εκτέλεσης για SQL API
sql_time = sql_end - sql_start

print(f"DataFrame API Execution Time: {df_time:.4f} seconds")
print(f"SQL API Execution Time: {sql_time:.4f} seconds")

#Σύγκριση χρόνων
if df_time < sql_time:
    print("DataFrame API was faster.")
else:
    print("SQL API was faster.")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+------------------+----+
|year|AREA NAME  |closed_case_rate  |rank|
+----+-----------+------------------+----+
|2010|Rampart    |32.947355855318136|1   |
|2010|Olympic    |31.962706191728422|2   |
|2010|Harbor     |29.63203463203463 |3   |
|2011|Olympic    |35.212167689161554|1   |
|2011|Rampart    |32.511779630300836|2   |
|2011|Harbor     |28.652205202015008|3   |
|2012|Olympic    |34.414818310523835|1   |
|2012|Rampart    |32.9464181029429  |2   |
|2012|Harbor     |29.815133276010318|3   |
|2013|Olympic    |33.52812271731191 |1   |
|2013|Rampart    |32.08287360549221 |2   |
|2013|Harbor     |29.16422459266206 |3   |
|2014|Van Nuys   |31.80567315834039 |1   |
|2014|West Valley|31.31198995605775 |2   |
|2014|Mission    |31.16279069767442 |3   |
|2015|Van Nuys   |32.64134698172773 |1   |
|2015|West Valley|30.27597402597403 |2   |
|2015|Mission    |30.179460678380153|3   |
|2016|Van Nuys   |31.880755720117726|1   |
|2016|West Valley|31.54798761609907 |2   |
|2016|Footh